# Lecture-1: Overview and Tokenization

In the first lecture of CS-336 we mainly studied about **tokenization** and with the focus of mainly at **BPE (Byte-Pair Encoding) tokenizer**. 

## 1. Intro To Tokenization

When we talk about Language Models we consider them like a giant math functions. They don't understand "words"; they understand numbers. We need a way to turn `The quick brown fox` into something like `[42, 512, 999, 204]`. This is where tokenization comes into play. Tokenization is the process of breaking a stream of raw text (like above example) into smaller and discrete units called tokens. <br>
A language model places a proabbility distribution over sequence of tokens. Hence, we need a procedure that encodes strings into tokens and also need a procedure that decodes tokens back into strings. A **tokenizer** is a class that implements the encode and decode methods.

### 1.1 Approach-1: Character Based Tokenization

Before fancy algorithms like BPE, WordPiece and Unigram came there existed the most fundamental tokenization method of all which was tokenizing text at the character level. In the layman terms this concept sounds too simple and that's the point of character based tokenization.<br>

In character based tokenization our token is literally a single character.<br>
For example the sentence:
```css
hello world
```

gets tokenized as:

```css
['h', 'e', 'l', 'l', 'o', ' ', 'w', 'o', 'r', 'l', 'd']
```

Every letter or punctuation mark, whitespace and symbol becomes its own token. That means the tokenizer’s vocabulary is simply:
- All letters (a–z, A–Z)
- All digits (0–9)
- All punctuation and symbols (!, @, #, $, …)
- All whitespace types
- Any special tokens you define (e.g., BOS, EOS)

So the vocabulary size might be ~100–300 tokens depending on the language hence the first drawback of this method is that it is is tiny in comparison.


#### Some strengths of this approach

1. **Zero Out-of-Vocabulary Issues**:

With word-level tokenizers if we encounter a rare word like:
```css
supercalifragilisticexpialidocious
```
then the model can’t represent it as a single token unless it was already in the vocabulary. But character level tokenization solves this probelm. It breaks the above word into:
```css
['s', 'u', 'p', 'e', 'r', ...]
```
and makes everything representable.

2. **Useful in Low-Data Settings**

If we are training a small RNN, a tiny GPT-like model from scratch(nanoGPT experiments), or a language model for a very narrow domain then small vocabulary helps the model converge faster and this is where character based approach is much useful. Karpathy’s early [char-RNNs for generating Shakespeare](https://github.com/karpathy/char-rnn) relied on this simplicity.

#### But Then What’s the Drawback?

1. **Sequence Length Explosion:**

Consider the below example:

```bash
hello world
```

If we see here clearly there are `11` character level tokens. Now imagine if we are doing some real life language modelling hence then we need real sentences, paragraphs or books and then model will need to process much longer sequences. Longer sequence will mean more memory, more training time, fewer tokens per batch (slower learning) and also more attention computation (quadratic cost). Hence **this is the single biggest reason modern LLMs don’t use pure character tokenization.**


### 1.2 Approach-2: Word Based Tokenization

After failure of character based tokenization failure the researchers thought that **A text is just a sequence of words. So what if we tokenize the text by splitting it into words.** This idea is very natural because that’s how humans think but then machines aren't humans and they don't think in words and that's why this mismatch turned word-based tokenization from NLP’s first “obvious solution” into one of its biggest limitations.<br>

#### What is Word-Based Tokenization?

In simple explanation word tokenization splits text at whitespace and punctuation into words. For example:<br>

```bash
Input:   "Hello, world! I’m learning NLP."
Tokens:  ["Hello", "world", "I’m", "learning", "NLP"]
```

The tokenizer removes punctuation or isolates it and then everything else becomes a word. After this each unique word becomes a vocabulary entry and then each word maps to an integer ID and finally model processes sequences of those integer IDs.

#### How This Approach Fails:

1. **Vocabulary Explosion:**

When every unique word becomes an entry in the vocabulary then even simple variations produce new tokens. For example words like below:

```css
apple
apples
Apple
APPLE
apple's
apple-like
```
will generate new tokens with even simple variations and this will result in size explosion.


2. **Out of Vocabulary (OOV):**

If a word never appeared during training the model cannot represent it. Hence the model can’t learn representations for words that don’t exist in training.

### 1.3 Approach-3: Byte Pair Encoding (BPE)

If you reached till here then now you must be wondering
- How do LLMs break text into tokens?
- How do they avoid exploding vocabulary sizes?
- How do they handle unknown words?

The answer of all these questions is **Byte Pair Encoding** which is one of the most influential algorithms in modern NLP tokenization.

#### Defining BPE

Basically BPE learns the most frequent pairs of symbols and merges them into longer symbols iteratively. First we beging with individual bytes or individual characters (depending on implementation) then repeatedly merge the most common adjacent pairs until our vocabulary reaches the target size.<br>
This results in common words become single tokens and rare words are decomposed into multiple subword chunks and due to this all texts become representable because we start from raw bytes.

#### Building BPE With Example

Consider the example:

```css
banana bandana ban
```

- **Step-1: Add word boundary markers**

We append `_` at the end of each word (this prevents accidental merges across word boundaries). Now our new corpus becomes:

```css
b a n a n a _
b a n d a n a _
b a n _
```

- **Step-2: Count Initial Pair Frequencies**

Now we break each word into pairs. The word `b a n a n a _` becomes pair like
```css
(b a), (a n), (n a), (a n), (n a), (a _)
```

Word `b a n d a n a _` becomes

```css
(b a), (a n), (n d), (d a), (a n), (n a), (a _)
```

Word `b a n _` becomes

```css
(b a), (a n), (n _)
```

The frequency of each word is as:

```css
| Pair    | Count |
| ------- | ----- |
| **b a** | 3     |
| **a n** | 5     |
| **n a** | 3     |
| **a _** | 2     |
| n d     | 1     |
| d a     | 1     |
| n _     | 1     |
```

As we can see above most frequent pair is `an`


- **Step-3: Apply First Merge ("an")**

Now we uppdate the corpus as:

a) Word 1: `b a n a n a _` which becomes `b an a n a _` and after merging second `a n`: `b an an a _`

b) Word 2: `b a n d a n a _` which becomes `b an d an a _`.

c) Word 3: `b a n _` which becomes `b an _`

So after first merge we have:

```css
b an an a _
b an d an a _
b an _
```


- **Step 4: Recount Pairs**

Now we will extract pairs again:

a) Word: `b an an a _` which pairs as

```css
(b an), (an an), (an a), (a _)
```

b) Word: `b an d an a _` now pairs as

```css
(b an), (an d), (d an), (an a), (a _)
```

c) Word: `b an _` pairs as

```css
(b an), (an _)
```
The table becomes now:

```css

| Pair     | Count |
| -------- | ----- |
| **b an** | 3     |
| **an a** | 2     |
| **a _**  | 2     |
| an an    | 1     |
| an d     | 1     |
| d an     | 1     |
| an _     | 1     |
```

The most frequent pair is now "ban" so the **second merge** will be **b + an = ban**.

- **Step 5: Apply Second Merge ("ban")**

Now reapply merge in all words:

a) Word 1: `b an an a _` becomes `ban an a _`

b) Word 2: `b an d an a _` becomes `ban d an a _`

c) Word 3: `b an _` becomes `ban _`


Now the updated corpus is

```css
ban an a _
ban d an a _
ban _
```

- **Step 6: Recount Pairs Again**

a) We have word 1 pairs as `(ban an), (an a), (a _)`.

b) Word 2 pairs as `(ban d), (d an), (an a), (a _)`

c) Word 3 pairs as `(ban _)`.


Now the count is as follows:


```css
| Pair     | Count |
| -------- | ----- |
| **an a** | 2     |
| **a _**  | 2     |
| ban an   | 1     |
| ban d    | 1     |
| d an     | 1     |
| ban _    | 1     |
```

After this merge we have count of two pairs tie as `an a` and `a _`. We can pick *either* but in practice BPE picks deterministically.<br>
First we pick `an a` to make `ana`. This is a very natural subword which is “ana” and common in “banana”.

- **Step 6: Apply Merge ("ana")**

After applying merge we update corpus as

a) Word 1: `ban an a _` becomes `ban ana _`

b) Word 2: `ban d an a _` becomes `ban d ana _`

c) But word 3 remains unchanged: `ban _`

Hence now new updated corpus us:

```css
ban ana _
ban d ana _
ban _
```

Which already looks very “natural” as original word `ban` + `ana` as *banana* and `ban` + `d` + `ana` → *bandana*


- **Step 7: Recount Pairs**

Now we have pairs as `(ban ana)`, `(ana _)`, `(ban d)`, `(d ana)`, `(ban _)` which has count of:

```bash
| Pair      | Count |
| --------- | ----- |
| **ana _** | 2     |
| ban ana   | 1     |
| ban d     | 1     |
| d ana     | 1     |
| ban _     | 1     |
```

with the most frequent word now `ana + _` as `ana_` which creates the full subword for ending “ana_”. But typically we stop merging once we get meaningful subunits e.g. 10k merges for a real tokenizer.

## 2. BPE Tokenizer (Minimal Working Example)

### 2.1 Training BPE

In [1]:
import re
from collections import Counter, defaultdict

In [2]:
def get_stats(tokens_list):
    pairs = Counter()# Count frequency of adjacent token pairs
    for tokens in tokens_list:
        for i in range(len(tokens) - 1):
            pairs[(tokens[i], tokens[i+1])] += 1# Incrementing the count for the token pair
    return pairs

In [3]:
def merge_tokens(tokens_list, merge_pair):
    new_list = []# List to hold the updated token lists
    bigram = " ".join(merge_pair)# Create a regex pattern to match the bigram
    pattern = re.compile(r'(?<!\S)' + re.escape(bigram) + r'(?!\S)')# Regex to match the bigram as a whole word

    for tokens in tokens_list:
        text = " ".join(tokens)# Join tokens into a single string
        merged = pattern.sub("".join(merge_pair), text)# Replace bigram with merged token
        new_list.append(merged.split())# Split back into tokens and add to new list
    return new_list

In [4]:
def train_bpe(corpus, num_merges=10):
    # Split corpus into space-separated words, adding a special end-of-word mark
    tokens_list = [[c for c in word] for word in corpus.split()]
    merges = []# List to hold the merge operations
    for _ in range(num_merges):
        stats = get_stats(tokens_list)# Get frequency of adjacent token pairs
        if not stats:
            break
        # Most frequent pair
        best = max(stats, key=stats.get)
        merges.append(best)

        # Merge it across corpus
        tokens_list = merge_tokens(tokens_list, best)

    return merges

### 2.2 Applying BPE

In [5]:
def bpe_encode(word, merges):
    tokens = [c for c in word]# Initial tokens are characters
    # Create a dictionary for quick lookup of merges
    merge_dict = {"".join(pair): pair for pair in merges}
    # Repeat merging until no more merges can be applied
    changed = True
    while changed:
        changed = False# Flag to check if any merge happened
        i = 0# Index to traverse tokens
        while i < len(tokens) - 1:
            pair = tokens[i] + tokens[i+1]
            if pair in merge_dict:# If this merged symbol was learned
                tokens[i:i+2] = [pair]# Merge them
                changed = True
            else:
                i += 1# Move to the next token

    return tokens

### 2.3 Decoding

In [6]:
def bpe_decode(tokens):
    output = ""# Initialize output string
    for t in tokens:
        output += t# Concatenate tokens
    return output

In [7]:
corpus = "banana bandana ban"
merges = train_bpe(corpus, num_merges=10)

In [8]:
print("Learned merges:")
for m in merges:
    print(m)

Learned merges:
('a', 'n')
('b', 'an')
('an', 'a')
('ban', 'ana')
('ban', 'd')
('band', 'ana')


In [10]:
print("\nEncoding 'banana':")
encoded = bpe_encode("banana", merges)
print(encoded)


Encoding 'banana':
['b', 'ana', 'n', 'a']


In [11]:
print("\nDecoding:")
print(bpe_decode(encoded))


Decoding:
banana
